In [2]:
!pip install lda

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 351 kB 5.1 MB/s 
     |████████████████████████████████| 99 kB 7.5 MB/s 


In [3]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # 다항분포 나이브 베이즈 모델
from sklearn.metrics import accuracy_score    # 정확도 계산

In [7]:
df = pd.read_csv('wat.csv')
df

,Unnamed: 0,rating,review,date,userName
0,0,5,개발자 및 경영진분들 꼭 보시라고 별5개 답니다????? 꼭 읽으세요. 방금 결제마...,2020-05-26 12:27:32,저내림
1,1,5,넷플릭스에서 왓챠로 넘어온지 2개월째입니당\n음 영화를 좋아하시는 분이라면 넷플릭스...,2020-04-06 06:40:28,dkqjfsqfe
2,2,5,업데이트 전으로 되돌리는 방법이 있나요?\n최신으로 업데이트 한 후 다 본 작품도 ...,2022-07-01 02:30:57,뾰롱로로
3,3,5,안녕하세요! 코로나때문에 너무 심심해서 넷플로 먼저 시작했는데 너무 볼 것이 없어 ...,2020-08-25 01:03:45,jsa38
4,4,5,맥북으로 보시려면 앱스토어에서 아이패드용 앱으로 다운 받으시면 됩니다~ 그리고 유튜...,2021-03-01 10:45:08,dkjfew
...,...,...,...,...,...
22300,20300,5,꿀댐,2018-08-30 20:13:10,Google 사용자
22301,20301,5,굿뜨,2019-08-30 12:35:15,Google 사용자
22302,20302,5,❤,2021-05-03 18:59:55,양지수
22303,20303,3,. ...,2020-04-22 15:54:03,Google 사용자


In [8]:
# 특정 단어가 들어간 리뷰 조회
df[df['review'].str.contains('버그') == True]['review']

185      아이폰 xs에서는 왓챠 못보나요? 핸드폰 바꿔서 재생하는데 sdk 오류라고 재생이 ...
286      며칠째 영상이 안열리고 화면이 튕깁니다. 오류 발생하는 시기만큼 기간 연장해주셔야죠...
693      안녕하세요 평소에도 저렴한 가격에 다양한 영화를 제공해주는 욋챠 덕분에 즐거운 영화...
779      왓챠 잘 이용하고 있는 고객인데요~ 앱이 실행하면 갑자기 꺼집니다. 들어가지질 않아...
888      동영상 보기도 좋고 오프닝 뛰어넘기 기능도 센스있어요. \n제발 듀라라라 들여와주세...
                               ...                        
19842    크롬캐스트 버그 그대로. 앱내에서 고객문의 해도 답변없음. 오히려 메일 문의라 마음...
20060                           징그러워서 못보겠네요.. 버그가 왜이리 많은지~
20271       버그가 약간씩 있긴하지만 금방 고쳐주실꺼라 생각해요 잘쓸께요! 크롬캐스트 잘됩니다!
20588    한달 써봤는데 (갤s7) 제 방이 와이파이가 두칸? 정도밖에안떠서 버퍼링걸릴줄일았는...
21364    처음엔 잘되다가 갑자기 관람등급 이후로 재생이 되질 않습니다. 아직 오픈 초기라 그...
Name: review, Length: 75, dtype: object

In [10]:
df[df['review'].str.contains('버그') == True]['rating']

185      1
286      1
693      5
779      4
888      5
        ..
19842    1
20060    2
20271    5
20588    4
21364    4
Name: rating, Length: 75, dtype: int64

In [11]:
# 평점이 4인 리뷰 조회
df[df['rating'] == 4]['review']

10       아니 베이직이용권을 구매해서 구입내역에도 7900원이 결제 됐다고 뜨는데 왜자꾸 들...
11       최소 16년도부터 이 버퍼링 문제때문에 불편을 제기하는 사람들이 많았는데 2020년...
22       홈버튼을 누르거나 앱 내에서 작은 화면 보기가 안되는게 조금 아쉬워요ㅠㅠ 그거 빼고...
69            전에는 괜찮았는데 오늘 맥북에 다운로드했던 왓챠앱이 몇번눌러도 창이안떠요 ㅠㅠㅠ
82       트루 디텍티브 시즌1 7화를 보던중에 멈췄다가 다시 \n켜보니까 자막이 안나오더니 ...
                               ...                        
22236                                                 자막퀄좀
22249                                                  B c
22275                                                 Goof
22282                                                  Wow
22286                                                 괜찮네여
Name: review, Length: 3714, dtype: object

In [12]:
data_list = df['review']
data_list

0        개발자 및 경영진분들 꼭 보시라고 별5개 답니다????? 꼭 읽으세요. 방금 결제마...
1        넷플릭스에서 왓챠로 넘어온지 2개월째입니당\n음 영화를 좋아하시는 분이라면 넷플릭스...
2        업데이트 전으로 되돌리는 방법이 있나요?\n최신으로 업데이트 한 후 다 본 작품도 ...
3        안녕하세요! 코로나때문에 너무 심심해서 넷플로 먼저 시작했는데 너무 볼 것이 없어 ...
4        맥북으로 보시려면 앱스토어에서 아이패드용 앱으로 다운 받으시면 됩니다~ 그리고 유튜...
                               ...                        
22300                                                   꿀댐
22301                                                   굿뜨
22302                                                    ❤
22303                                                . ...
22304                                                 👍👍👍👍
Name: review, Length: 22305, dtype: object

In [16]:
# 4,5 : 긍정 / 1,2 : 부정
pos_list = df[df['rating'] >= 4]
neg_list = df[df['rating'] <= 2]

In [17]:
# 각각 다른 배열로 저장
pos_corpus = np.array(pos_list['review'])
neg_corpus = np.array(neg_list['review'])

In [18]:
# 두 데이터를 다시 통합
total_data = np.concatenate((pos_corpus,neg_corpus))

In [19]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [824 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [85.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bioni

In [20]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [21]:
from konlpy.tag import Okt
import nltk

In [22]:
okt=Okt()

In [23]:
# 리뷰들을 모두 한 문자열로 통합
reviews = ''
   
for i in df['review'] : 
  reviews = reviews + i + '\n'

In [24]:
tokens = okt.morphs(reviews)

In [25]:
result = nltk.Text(tokens)

In [26]:
review_pos = okt.pos(reviews)

In [27]:
pos_df = pd.DataFrame(review_pos)
pos_df.columns = ['token', 'pos']
pos_df.head()

,token,pos
0,개발자,Noun
1,및,Noun
2,경,Modifier
3,영진,Noun
4,분들,Suffix


In [28]:
noun = pos_df[pos_df['pos']== 'Noun']['token']
josa = pos_df[pos_df['pos']== 'Josa']['token']
verbprefix = pos_df[pos_df['pos']== 'Verbprefix']['token']
verb = pos_df[pos_df['pos']== 'Verb']['token']
punctuation = pos_df[pos_df['pos']== 'Punctuation']['token']
adjective = pos_df[pos_df['pos']== 'Adjective']['token']
suffix = pos_df[pos_df['pos']== 'Suffix']['token']
number = pos_df[pos_df['pos']== 'Number']['token']
adverb = pos_df[pos_df['pos']== 'Adverb']['token']
conjunction = pos_df[pos_df['pos']== 'Conjunction']['token']
alpha = pos_df[pos_df['pos']== 'Alpha']['token']
foreign = pos_df[pos_df['pos']== 'Foreign']['token']
determiner = pos_df[pos_df['pos']== 'Determiner']['token']
modifier = pos_df[pos_df['pos']== 'Modifier']['token']
koreanparticle = pos_df[pos_df['pos']== 'KoreanParticle']['token']
eomi = pos_df[pos_df['pos']== 'Eomi']['token']
exclamation = pos_df[pos_df['pos']== 'Exclamation']['token']
url = pos_df[pos_df['pos']== 'URL']['token']
hashtag = pos_df[pos_df['pos']== 'Hashtag']['token']
email = pos_df[pos_df['pos']== 'Email']['token']
preeomi = pos_df[pos_df['pos']== 'PreEomi']['token']
screenname = pos_df[pos_df['pos']== 'ScreenName']['token']

In [29]:
noun_list = noun.to_list()

In [30]:
# 불용어 처리

In [31]:
# total_data = 리뷰들을 배열로 처리해놓은 것
total_data.shape

(19968,)

In [32]:
c_vetorizer = CountVectorizer(analyzer='word')
count = c_vetorizer.fit_transform(noun_list)

In [33]:
model = lda.LDA(n_topics = 4, n_iter = 1000)
model.fit(count)

In [38]:
topic_vocab = c_vetorizer.get_feature_names()
topic_word = model.topic_word_
n_top_word = 300
dist = []
for i, topic_dist in enumerate(topic_word):
    dist.append(topic_dist)
    topic_words = np.array(topic_vocab)[np.argsort(topic_dist)][:-n_top_word:-1]
    print('Topic', i+1, topic_words)

Topic 1 ['결제' '환불' '보고' '작품' '정말' '무료' '보기' '제발' '갑자기' '개선' '사람' '소리' '리뷰' '버퍼링'
 '자막' '처음' '가입' '검색' '별로' '왓챠' '때문' '선택' '시즌' '메일' '자체' '조절' '핸드폰' '구매'
 '영어' '크롬캐스트' '혹시' '이유' '옛날' '취향' '모바일' '사용자' '아이폰' '버전' '제대로' '장르' '다만'
 '시작' '기기' '발생' '결재' '만료' '거의' '이상' '종류' '갤럭시' '화면' '최근' '사랑' '어플' '안드로이드'
 '탈퇴' '가끔' '구글' '마음' '인증' '프로그램' '어제' '한국어' '유저' '네트워크' '게임' '느낌' '비교'
 '외국' '찾기' '동안' '기본' '프리미엄' '알림' '변경' '정액' '시도' '된거' '공부' '국가' '오프닝' '일주일'
 '심지어' '거리' '부터' '플랫폼' '마이' '덕분' '매번' '전부' '매달' '한가지' '국내' '화가' '키즈' '대한'
 '사이트' '프로' '실망' '표시' '수도' '번호' '생기' '자료' '설명' '이용권' '얼른' '몇개' '웨이브' '굳이'
 '정상' '가기' '실패' '이건' '장난' '불가' '필요' '편의' '순간' '예상' '통신' '마지막' '하니' '왓챠플레'
 '도저히' '잠깐' '초기' '편이' '메뉴' '어이' '발전' '티빙' '아쉬움' '약간' '저기' '내야' '쓰기' '비번'
 '후회' '워킹데드' '문구' '중이' '면서' '인생' '한국영' '거기' '살짝' '불편' '밝기' '이틀' '종종' '완료'
 '중드' '먹통' '스트레스' '다해' '크게' '네이버' '어찌' '캡쳐' '크레딧' '극장판' '통합' '달라' '방송'
 '플렉스' '는걸' '대사' '피씨' '체르노빌' '의미' '무제한' '셜록' '소비자' '지난' '대비' '가족' '학생'
 '비용' '지도' '페이' '엔딩' '알람' '페이스북' '이야기' '계획

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
model.topic_word_

array([[2.64425047e-07, 2.64425047e-07, 2.64425047e-07, ...,
        2.64425047e-07, 2.64425047e-07, 2.67069298e-05],
       [2.66057218e-07, 1.59900388e-04, 2.66057218e-07, ...,
        2.66057218e-07, 2.68717790e-05, 2.66057218e-07],
       [2.64915399e-07, 2.64915399e-07, 7.86825227e-03, ...,
        2.64915399e-07, 2.64915399e-07, 2.64915399e-07],
       [2.69477773e-05, 2.66809677e-07, 2.66809677e-07, ...,
        2.69477773e-05, 2.66809677e-07, 2.66809677e-07]])

In [39]:
stopword=['디즈니','왜','좀','수','볼','것','그냥','슝','더','땡','보고','점','거','진짜','요','다른','안','이','제','저','그','등','내','시','개','전','폰','뭉탱이','바로',
'다시','달','계속','아주','경우','뭐','정도','일단','앞','한번','때문','못','게','도','언제','편','레이디','유리',
'나','중','함','다만','정','이건','꼭','디','분','외','걸','흠','등등','후','로','완전','이제','그거','매우','건','여기','애','또','지금','은','아니','있다가','입니다','있는','같아요','있어서','부탁드립니다','좋겠습니다','있는데','같습니다','같은','있습니다','있게','있었으면','있어','있고',
'어떻게','같네요']

In [40]:
from collections import defaultdict
from pandas import read_table
import numpy as np
import math

class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def load_corpus(self, path):
        corpus = read_table(path, sep=',', encoding='utf-8')
        corpus = np.array(corpus)
        return corpus

    def count_words(self, training_set):
        # 학습데이터는 영화리뷰 본문(doc), 평점(point)으로 구성

        counts = defaultdict(lambda : [0, 0])
        for doc, point in training_set:
            # 영화리뷰가 text일 때만 카운트
            if self.isNumber(doc) is False:
                  
                  words = okt.pos(doc)

                  ls = []
                  for i,j in words : 
                    if j == 'Noun' or j == 'Adjective' or j == 'Verb' :
                        ls.append(i)

                  ls2 = []
                  for i in ls :
                    if i not in stopword :
                      ls2.append(i)

                  for word in ls2:
                    counts[word][0 if point > 3.5 else 1] += 1
        
        return counts

    def isNumber(self, s):
        try:
            float(s)
            return True
        except ValueError:
            return False

    def word_probabilities(self, counts, total_class0, total_class1, k):
        # 단어의 빈도수를 [단어, p(w|긍정), p(w|부정)] 형태로 반환
        return [(w,
                 (class0 + k) / (total_class0 + 2*k),
                 (class1 + k) / (total_class1 + 2*k))
                for w, (class0, class1) in counts.items()]

    def class0_probability(self, word_probs, doc):
        # 별도 토크나이즈 안하고 띄어쓰기로만
        docwords = okt.pos(doc)

        ls = []
        for i,j in docwords : 
          if j == 'Noun' or j == 'Adjective' or j == 'Verb' :
              ls.append(i)

        ls2 = []
        for i in ls :
          if i not in stopword :
            ls2.append(i)

        docwords = ls2

        # 초기값은 모두 0으로 처리
        log_prob_if_class0 = log_prob_if_class1 = 0.0

        # 모든 단어에 대해 반복
        for word, prob_if_class0, prob_if_class1 in word_probs:
            # 만약 리뷰에 word가 나타나면
            # 해당 단어가 나올 log 확률을 더해 줌
            if word in docwords:
                log_prob_if_class0 += math.log(prob_if_class0)
                log_prob_if_class1 += math.log(prob_if_class1)

            # 만약 리뷰에 word가 나타나지 않는다면
            # 해당 단어가 나오지 않을 log 확률을 더해 줌
            # 나오지 않을 확률은 log(1-나올 확률)로 계산
            else:
                log_prob_if_class0 += math.log(1.0 - prob_if_class0)
                log_prob_if_class1 += math.log(1.0 - prob_if_class1)

        prob_if_class0 = math.exp(log_prob_if_class0)
        prob_if_class1 = math.exp(log_prob_if_class1)
        return prob_if_class0 / (prob_if_class0 + prob_if_class1)

    def train(self, trainfile_path):
        training_set = self.load_corpus(trainfile_path)

        # 범주0(긍정)과 범주1(부정) 문서 수를 세어 줌
        num_class0 = len([1 for _, point in training_set if point > 3.5])
        num_class1 = len(training_set) - num_class0

        # train
        word_counts = self.count_words(training_set)
        self.word_probs = self.word_probabilities(word_counts,
                                                  num_class0,
                                                  num_class1,
                                                  self.k)

    def classify(self, doc):
        return self.class0_probability(self.word_probs, doc)

In [41]:
doc = '자막이 왜 개선 안되나요'

In [42]:
docwords = doc.split()
docwords

['자막이', '왜', '개선', '안되나요']

In [44]:
docwords = okt.pos(doc)

ls = []
for i,j in docwords : 
  if j == 'Noun' or j == 'Adjective' or j == 'Verb' :
      ls.append(i)

ls2 = []
for i in ls :
  if i not in stopword :
    ls2.append(i)

docwords = ls2
docwords

['자막', '개선', '안되나요']

In [45]:
from IPython.core.display import DisplayObject
DisplayObject

IPython.core.display.DisplayObject

In [46]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [47]:
stopword=['디즈니','왜','좀','수','볼','것','그냥','슝','더','땡','보고','점','거','진짜','요','다른','안','이','제','저','그','등','내','시','개','전','폰','뭉탱이','바로',
'다시','달','계속','아주','경우','뭐','정도','일단','앞','한번','때문','못','게','도','언제','편','레이디','유리',
'나','중','함','다만','정','이건','꼭','디','분','외','걸','흠','등등','후','로','완전','이제','그거','매우','건','여기','애','또','지금','은','아니','있다가','입니다','있는','같아요','있어서','부탁드립니다','좋겠습니다','있는데','같습니다','같은','있습니다','있게','있었으면','있어','있고',
'어떻게','같네요','하는','할','될','몇','가지','바','바','남깁니다','볼때','듭니','사이','있도록','바랄게요']

In [48]:
df = df[['review', 'rating']]

In [49]:
df.to_csv('filtered.csv',index=None,header=None)

In [50]:
model = NaiveBayesClassifier()
model.train(trainfile_path='filtered.csv')

In [51]:
model.classify('나는 넷플릭스로 갈아탑니다. 디즈니는 컨텐츠도 별로 없고 자막 해석도 구려요. 진짜 싫어요. 문제가 많습니다')

0.08586085401844577

In [52]:
# 긍정 단어 가중치로 딕셔너리 만들기
dict_p = {}
for i in model.word_probs:
    dict_p[i[0]] = i[1]
# 부정 단어 가중치로 딕셔너리 만들기    
dict_n = {}
for i in model.word_probs:
    dict_n[i[0]] = i[2]
# 각 딕셔너리 값을 내림차순 정렬
sort_dict_p = sorted(dict_p.items(), key=operator.itemgetter(1), reverse=True)
sort_dict_n = sorted(dict_n.items(), key=operator.itemgetter(1), reverse=True)

In [56]:
# 각 리뷰별 상위 단어 추출
for i in sort_dict_p[:200]:
    print(i[0])  

영화
좋아요
왓챠
결제
잘
드라마
앱
자막
기능
넷플릭스
사용
정말
좋은
작품
굿
영상
무료
때
화면
컨텐츠
해주세요
업데이트
많아서
재생
좋은데
이용
좋겠어요
어플
좋네요
최고
했는데
추가
넷플
해서
화질
보기
환불
플레이
제발
합니다
돈
좋습니다
생각
많고
많은
추천
없는
좋음
배속
이용권
있어요
감사합니다
하면
없어서
해
있으면
해지
굳
한국
사람
구독
설정
오류
미드
가격
다양한
갑자기
취소
문제
개선
쓰고
시즌
애니
넘
많아요
좋고
리뷰
영어
시리즈
보는
한
조금
자동
지원
티비
처음
시청
시간
선택
서비스
해결
하나
검색
콘텐츠
취향
조절
다운
보고싶은
된
다운로드
좋
하는데
카테고리
최신
혹시
부탁드려요
더빙
없네요
별로
부분
오늘
장르
보려고
버퍼링
삭제
하고
다음
사랑
가요
말
많네요
되면
되는
이번
자꾸
번
좋겠네요
없고
좋아하는
올려주세요
체험
해도
가입
옛날
애니메이션
보는데
좋을
되고
구매
보면
이어
개인
요즘
해외
방법
별
본
확인
아이폰
알
아이패드
예능
해리포터
개월
하기
좋다
보고싶어요
자주
종류
안되네요
소리
빠른
보고있어요
불편해요
실행
없어요
프로필
봤는데
되어
이런
연결
기
볼수
있네요
뒤
보다가
아쉬운
주
건지
가끔
감상
저장
핸드폰
예전
되는데
중간
재밌게
하려고
안녕하세요
가
버튼
마음
자체
아쉬워요
계정
써
로그인
최근
버전
듯


In [57]:
for i in sort_dict_n[0:200]:
    print(i[0])

결제
영화
재생
앱
왓챠
환불
무료
화면
영상
돈
했는데
해주세요
해지
사용
업데이트
오류
자막
넷플릭스
해서
어플
문제
갑자기
이용권
드라마
이용
때
취소
자동
잘
삭제
기능
작품
하면
해도
없고
구독
보기
자꾸
화질
소리
해결
제발
컨텐츠
없네요
생각
버퍼링
합니다
개선
하나
해
티비
안되고
로딩
넷플
연결
고객
건지
안되네요
플레이
정말
하는데
번
가입
문의
리뷰
확인
보는데
되고
실행
설치
접속
오늘
센터
가요
사람
로그인
서비스
메일
다운
별로
아예
없는
좋은데
시청
한
배속
하고
처음
해야
검색
이어
되는
말
와이파이
개월
설정
된
정기
되는데
없음
답변
자체
뜨고
체험
핸드폰
없어요
나오고
다음
하지
이런
보면
건가
렉
없어서
안됩니다
선택
이번
최악
돼요
요청
버튼
같은데
무슨
계정
아니고
재
주
하려고
안됨
시간
좋겠어요
현상
않고
보려고
일
이유
방법
데이터
제대로
다운로드
동영상
수정
크롬캐스트
보는
부분
없는데
임
빠른
최신
구매
됨
되지
추가
하기
모바일
한국
내고
보다가
만료
불편해요
기기
결재
신청
중간
별
서버
발생
추천
콘텐츠
뒤
탈퇴
카테고리
본
기간
문자
유료
업뎃
사용자
구글
갤럭시
창
자주
처리
지원
이후
뜨네요
조절
보니
이메일
고쳐주세요
상태
하루
됩니다
누르면
인터넷
되나요
더빙
불편합니다
되네요
주고


In [58]:
ps_app = ['좋아요','굿', '좋네요', '최고', '좋습니다', '좋음', '좋고', '좋다' ]

In [59]:
ps_function = ['자막', '기능', '사용', '화면', '업데이트', '재생', '화질',
               '배속', '설정', '개선', '리뷰', '영어', '지원', '티비', '시청',
               '시간', '조절', '다운', '다운로드', '카테고리', '더빙', '버퍼링',
               '체험', '소리', '빠른', '연결', '버전']

In [60]:
ps_contents = ['영화', '넷플릭스', '작품', '컨텐츠', '추천', '이용권', '한국', '미드',
               '다양한', '시즌', '애니', '많아요', '시리즈', '콘텐츠', '취향', '최신',
               '장르', '사랑', '가입', '구매', '개인', '예능', '해리포터', '종류', '재밌게', '최근']

In [61]:
ps_payment = ['결제', '무료', '환불', '돈', '해지', '구독', '가격', '개월']

In [62]:
ng_app = ['앱','어플' ,'오류','플러스','최악','별로','없네요','안되네요','리뷰',
          '무슨','없어요','쓰레기','불편해요','실망','불편함','먹통','개판','불편합니다','불편하네요',
          '불편한','않아요']

In [63]:
ps_function = [ '자막' , '화면' ,'기능','영상' ,'설정','배속','번역','조절','로딩','영화',
               '인증','밝기','서비스','성인','더빙','접속','지원','한국어','업데이트','로그인',
               '설치','크롬캐스트','속도','버튼','다운','다운로드','화질','연결','한글','저장',
               '검색','모바일','싱크','소리','모드','팝업','안드로이드','아이패드','갤럭시','오역',
               '버그','렉','태블릿']

In [64]:
ps_contents = [ '컨텐츠','마블','설강화','콘텐츠','드라마','티비','작품','시즌','시리즈',
               '역사왜곡','회차','차별','애니메이션']

In [65]:
ps_payment = [ '결제' ,'돈','구독','가입','취소','환불','해지','무료','카드','구매','탈퇴']

In [67]:
ps_app_dic = {}
for topic in ps_app:
    for word,value in dict_p.items():
        if(topic == word):
            ps_app_dic[word] = value
print(ps_app_dic)

{'좋아요': 0.15310884549109707, '굿': 0.047350660539919584, '좋네요': 0.0356476163124641, '최고': 0.034427053417576105, '좋습니다': 0.02466255025847214, '좋음': 0.022006031016657095, '좋고': 0.015328834003446295, '좋다': 0.008723434807581849}


In [71]:
ps_function_dic = {}
for topic in ps_app:
    for word,value in dict_p.items():
        if(topic == word):
            ps_function_dic[word] = value
print(ps_function_dic)

{'좋아요': 0.15310884549109707, '굿': 0.047350660539919584, '좋네요': 0.0356476163124641, '최고': 0.034427053417576105, '좋습니다': 0.02466255025847214, '좋음': 0.022006031016657095, '좋고': 0.015328834003446295, '좋다': 0.008723434807581849}


In [72]:
ps_contents_dic = {}
for topic in ps_app:
    for word,value in dict_p.items():
        if(topic == word):
            ps_contents_dic[word] = value
print(ps_contents_dic)

{'좋아요': 0.15310884549109707, '굿': 0.047350660539919584, '좋네요': 0.0356476163124641, '최고': 0.034427053417576105, '좋습니다': 0.02466255025847214, '좋음': 0.022006031016657095, '좋고': 0.015328834003446295, '좋다': 0.008723434807581849}


In [73]:
ps_payment_dic = {}
for topic in ps_app:
    for word,value in dict_p.items():
        if(topic == word):
            ps_payment_dic[word] = value
print(ps_payment_dic)

{'좋아요': 0.15310884549109707, '굿': 0.047350660539919584, '좋네요': 0.0356476163124641, '최고': 0.034427053417576105, '좋습니다': 0.02466255025847214, '좋음': 0.022006031016657095, '좋고': 0.015328834003446295, '좋다': 0.008723434807581849}


In [78]:
ng_app_dic = {}
for topic in ps_app:
    for word,value in dict_n.items():
        if(topic == word):
            ng_app_dic[word] = value
print(ng_app_dic)

{'좋아요': 0.006385772260682741, '굿': 0.00101456194795894, '좋네요': 0.0017307233229887802, '최고': 0.0030436858438768204, '좋습니다': 0.0020888040105037, '좋음': 0.0018500835521604202, '좋고': 0.00220816423967534, '좋다': 0.00017904034375746002}


In [79]:
ng_function_dic = {}
for topic in ps_app:
    for word,value in dict_n.items():
        if(topic == word):
            ng_function_dic[word] = value
print(ng_function_dic)

{'좋아요': 0.006385772260682741, '굿': 0.00101456194795894, '좋네요': 0.0017307233229887802, '최고': 0.0030436858438768204, '좋습니다': 0.0020888040105037, '좋음': 0.0018500835521604202, '좋고': 0.00220816423967534, '좋다': 0.00017904034375746002}


In [80]:
ng_contents_dic = {}
for topic in ps_app:
    for word,value in dict_n.items():
        if(topic == word):
            ng_contents_dic[word] = value
print(ng_contents_dic)

{'좋아요': 0.006385772260682741, '굿': 0.00101456194795894, '좋네요': 0.0017307233229887802, '최고': 0.0030436858438768204, '좋습니다': 0.0020888040105037, '좋음': 0.0018500835521604202, '좋고': 0.00220816423967534, '좋다': 0.00017904034375746002}


In [81]:
ng_payment_dic = {}
for topic in ps_app:
    for word,value in dict_n.items():
        if(topic == word):
            ng_payment_dic[word] = value
print(ng_payment_dic)

{'좋아요': 0.006385772260682741, '굿': 0.00101456194795894, '좋네요': 0.0017307233229887802, '최고': 0.0030436858438768204, '좋습니다': 0.0020888040105037, '좋음': 0.0018500835521604202, '좋고': 0.00220816423967534, '좋다': 0.00017904034375746002}


In [87]:
def positive_review(doc):
    doc = doc
    app = 0
    function = 0
    contents = 0
    payment = 0
    total = 0




    words = okt.pos(doc)

    ls = []
    for i,j in words : 
      if j == 'Noun' or j == 'Adjective' or j == 'Verb' :
          ls.append(i)

    ls2 = []
    for i in ls :
      if i not in stopword :
        ls2.append(i)
    
    words = ls2

    for word in words:
        for _,i in ps_app_dic.items():
            if (word == _):
                app += i

        for _,i in ps_app_dic.items():
            if (word == _):
                function += i

        for _,i in ps_app_dic.items():
            if (word == _):
                contents += i

        for _,i in ps_app_dic.items():
            if (word == _):
                payment += i
    total = app + function + contents + payment
    print('------------------------------------------------------리뷰 감성 분석------------------------------------------------------')
    print('-------------------------------------------------------토픽별 점수--------------------------------------------------------')
    print('앱 총평  : '+str(app)+'  | 기능 : '+str(function)+'  | 콘텐츠 : '+str(contents)+'  | 결제 : '+str(payment))
    print('총 합계 : '+str(total))
    print('--------------------------------------------------------------------------------------------------------------------------')  

In [88]:
positive_review('이 앱은 사용성이 좋습니다. 오리지널 콘텐츠의 종류가 다양합니다. 또한 자막지원이 훌륭합니다. 이 앱 추천합니다')

------------------------------------------------------리뷰 감성 분석------------------------------------------------------
-------------------------------------------------------토픽별 점수--------------------------------------------------------
앱 총평  : 0.02466255025847214  | 기능 : 0.02466255025847214  | 콘텐츠 : 0.02466255025847214  | 결제 : 0.02466255025847214
총 합계 : 0.09865020103388857
--------------------------------------------------------------------------------------------------------------------------
